In [2]:
import azure.core
from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [20]:
# https://www.youtube.com/watch?v=eIS3ZAVIgn8&ab_channel=BinodSumanAcademy
resource_name = "CS222"
workspace_name = "CS222-VSCode"
subscriptionID = "488c3044-7c03-4839-be5f-8ec3ee9c0ce4" # Please enter your subscription id here
# aml_compute_target = "demo-cluster" #ALL SMALL LETTER, NO underscore, 16ws long only.
aml_compute_target = "demo-cluster-2" #ALL SMALL LETTER, NO underscore, 16ws long only.
experiment_name= 'demo_expirement'

In [4]:
# Now create Workspace
try:
    ws=Workspace.from_config()
    print('Workspace is already exist')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_name,
                    create_resource_group=True,
                    subscription_id=subscriptionID,
                    location="East US 2")
    ws.write_config('.azureml')

Workspace is already exist


In [5]:
Workspace.from_config()

Workspace.create(name='CS222-SentimentAnalysis', subscription_id='488c3044-7c03-4839-be5f-8ec3ee9c0ce4', resource_group='CS222')

In [22]:
# Create Compute Target
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
#     vmsize = "STANDARD_D2_V2" # CPU
    vmsize = "STANDARD_NC6" # GPU
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vmsize,
                                                                min_nodes = 1, 
                                                                max_nodes = 2,
                                               idle_seconds_before_scaledown=1000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")

creating new compute target : demo-cluster-2
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded................................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached now


In [16]:
# Create environment to execute your code
env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml")
# env = Environment.get(workspace=ws,name="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu")
# to use given envs will have to import os and then os.system(pip install package)

In [9]:
exp = Experiment(ws, experiment_name)

In [26]:
config=ScriptRunConfig(source_directory="./",script="final_model_train.py",compute_target=aml_compute_target,environment=env)
execution=exp.submit(config)
# execution.wait_for_completion(show_output=True)

Submitting /mnt/e/yash/uiuc/Fall22/CS222/course-project-group-38/ml_model directory for run. The size of the directory >= 25 MB, so it can take a few minutes.
